# Storing and Querying Vector Data Using Similarity Search

## Tech Stack Used
#### SingleStore
#### LangChain

Note: Before starting with the code, make sure you have created a workspace and a database.

### Install LangChain and SingleStore DB python connector

In [12]:
!pip install --upgrade --quiet  singlestoredb

In [16]:
!pip install -qU langchain-community --quiet

### We will use OpenAI embeddings, so mention the OpenAI API Key

In [17]:
import os

# Set the environment variable for the OpenAI API key
os.environ["OPENAI_API_KEY"] = "Mention Your OpenAI API Key"

In [18]:
from langchain_community.vectorstores import SingleStoreDB
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

### Load some artificial data

In [19]:
# loading docs
# we will use some artificial data for this example
docs = [
    Document(
        page_content="""In the parched desert, a sudden rainstorm brought relief,
            as the droplets danced upon the thirsty earth, rejuvenating the landscape
            with the sweet scent of petrichor.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""Amidst the bustling cityscape, the rain fell relentlessly,
            creating a symphony of pitter-patter on the pavement, while umbrellas
            bloomed like colorful flowers in a sea of gray.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""High in the mountains, the rain transformed into a delicate
            mist, enveloping the peaks in a mystical veil, where each droplet seemed to
            whisper secrets to the ancient rocks below.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""Blanketing the countryside in a soft, pristine layer, the
            snowfall painted a serene tableau, muffling the world in a tranquil hush
            as delicate flakes settled upon the branches of trees like nature's own 
            lacework.""",
        metadata={"category": "snow"},
    ),
    Document(
        page_content="""In the urban landscape, snow descended, transforming
            bustling streets into a winter wonderland, where the laughter of
            children echoed amidst the flurry of snowballs and the twinkle of
            holiday lights.""",
        metadata={"category": "snow"},
    ),
    Document(
        page_content="""Atop the rugged peaks, snow fell with an unyielding
            intensity, sculpting the landscape into a pristine alpine paradise,
            where the frozen crystals shimmered under the moonlight, casting a
            spell of enchantment over the wilderness below.""",
        metadata={"category": "snow"},
    ),
]

embeddings = OpenAIEmbeddings()

### Store the vector data in our database

In [23]:
# Setup connection url as environment variable
os.environ["SINGLESTOREDB_URL"] = "admin:7wKMmtf9dCudBZHmLhitVTSTHcBMCeyn@svc-98ef7484-559e-498d-b400-efdec97e2064-dml.aws-london-1.svc.singlestore.com:3306/sampledb"

# Load documents to the store
docsearch = SingleStoreDB.from_documents(
    docs,
    embeddings,
    table_name="pavtab",  # use table with a custom name
)

### Ask a query

In [24]:
query = "trees in the snow"
docs = docsearch.similarity_search(query)  # Find documents that correspond to the query
print(docs[0].page_content)

In the urban landscape, snow descended, transforming
            bustling streets into a winter wonderland, where the laughter of
            children echoed amidst the flurry of snowballs and the twinkle of
            holiday lights.


SingleStoreDB elevates search capabilities by enabling users to enhance and refine search results through prefiltering based on metadata fields. 

In [25]:
query = "trees branches"
docs = docsearch.similarity_search(
    query, filter={"category": "snow"}
)  # Find documents that correspond to the query and has category "snow"
print(docs[0].page_content)

Blanketing the countryside in a soft, pristine layer, the
            snowfall painted a serene tableau, muffling the world in a tranquil hush
            as delicate flakes settled upon the branches of trees like nature's own 
            lacework.


In [27]:
query = "landscape"
docs = docsearch.similarity_search(query)  # Find documents that correspond to the query
print(docs[0].page_content)

In the urban landscape, snow descended, transforming
            bustling streets into a winter wonderland, where the laughter of
            children echoed amidst the flurry of snowballs and the twinkle of
            holiday lights.
